In [1]:
import re
import csv
import tqdm
import pandas as pd

import time
import datetime

from bs4 import BeautifulSoup
from selenium import webdriver as wbd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 1. Holiday
- 네이버달력 크롤링하는 방식
    - 한달 단위로 크롤링하며 붉은색(비영업일)에 해당하는 day 추출
    - 토요일은 검은색으로 나타나므로 별도로 추가해줄 것

In [2]:
options = wbd.ChromeOptions()
options.add_argument("headless")
options.add_argument("disable-gpu")
driver = wbd.Chrome("./chromedriver.exe", options=options)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

html = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EB%84%A4%EC%9D%B4%EB%B2%84%EB%8B%AC%EB%A0%A5&oquery=%EB%84%A4%E3%85%A3%E3%85%87%EB%B2%84%EB%8B%AC%EB%A0%A5&tqi=i4EPmlprvmZsslbDSkdssssssU0-356245'
driver.get(html)

C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\4121251317.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wbd.Chrome("./chromedriver.exe", options=options)


In [4]:
# 오늘부터 2000.1.1까지 개월 수 계산
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

# arguments
end_date= datetime.datetime(2023,6,30) # 마지막 날짜
start_date = datetime.datetime(2012,1,1) # 시작 날짜

diff_month = diff_month(end_date, start_date)
diff_month

137

In [5]:
start = time.time()  # 시작 시간 저장
holiday_list = []

for _ in range(diff_month): # argument로 개월 수 입력

    html = BeautifulSoup(driver.page_source, 'lxml')
    txt = html.find_all(class_ = 'cld_day')
    yyyymm = html.find_all('span', class_='_title')[0].get_text()
    print(f'[{yyyymm}]')


    for i, v in enumerate(zip(driver.find_elements(By.CLASS_NAME, 'cld_day'), txt)):
        tmp_list = []
        color = re.sub(r'[^0-9]', '', v[0].value_of_css_property('color'))
        day = v[1].string

        if color == '2312312311' or color == '3636361': # 회색, 검은색은 skip
            continue 
            
        print(f'''idx: {i}, color: {color}, day: {day}''')
        year = int(yyyymm[:4])
        month = int(yyyymm[5:7].lstrip("0"))
        day = int(day.lstrip("0"))
        # print(year, month, day)
        # print(datetime.datetime(year, month, day))
        tmp_list.append(day)
        holiday_list.append(datetime.datetime(year, month, day))
        
    # 한달 전으로 이동
    driver.find_element(By.XPATH, value = '//*[@id="direct_calendar"]/div/div[1]/div[1]/a[2]').click()
    time.sleep(1)
    
print("\n[elapsed time]:", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

[2023.06]
idx: 7, color: 25276781, day: 4
idx: 9, color: 25276781, day: 6
idx: 14, color: 25276781, day: 11
idx: 21, color: 25276781, day: 18
idx: 28, color: 25276781, day: 25
[2023.05]
idx: 5, color: 25276781, day: 5
idx: 7, color: 25276781, day: 7
idx: 14, color: 25276781, day: 14
idx: 21, color: 25276781, day: 21
idx: 27, color: 25276781, day: 27
idx: 28, color: 25276781, day: 28
idx: 29, color: 25276781, day: 29
[2023.04]
idx: 7, color: 25276781, day: 2
idx: 14, color: 25276781, day: 9
idx: 21, color: 25276781, day: 16
idx: 28, color: 25276781, day: 23
idx: 35, color: 25276781, day: 30
[2023.03]
idx: 3, color: 25276781, day: 1
idx: 7, color: 25276781, day: 5
idx: 14, color: 25276781, day: 12
idx: 21, color: 25276781, day: 19
idx: 28, color: 25276781, day: 26
[2023.02]
idx: 7, color: 25276781, day: 5
idx: 14, color: 25276781, day: 12
idx: 21, color: 25276781, day: 19
idx: 28, color: 25276781, day: 26
[2023.01]
idx: 0, color: 25276781, day: 1
idx: 7, color: 25276781, day: 8
idx: 14, 

idx: 28, color: 25276781, day: 26
idx: 29, color: 25276781, day: 27
[2019.12]
idx: 0, color: 25276781, day: 1
idx: 7, color: 25276781, day: 8
idx: 14, color: 25276781, day: 15
idx: 21, color: 25276781, day: 22
idx: 24, color: 25276781, day: 25
idx: 28, color: 25276781, day: 29
[2019.11]
idx: 7, color: 25276781, day: 3
idx: 14, color: 25276781, day: 10
idx: 21, color: 25276781, day: 17
idx: 28, color: 25276781, day: 24
[2019.10]
idx: 4, color: 25276781, day: 3
idx: 7, color: 25276781, day: 6
idx: 10, color: 25276781, day: 9
idx: 14, color: 25276781, day: 13
idx: 21, color: 25276781, day: 20
idx: 28, color: 25276781, day: 27
[2019.09]
idx: 0, color: 25276781, day: 1
idx: 7, color: 25276781, day: 8
idx: 11, color: 25276781, day: 12
idx: 12, color: 25276781, day: 13
idx: 13, color: 25276781, day: 14
idx: 14, color: 25276781, day: 15
idx: 21, color: 25276781, day: 22
idx: 28, color: 25276781, day: 29
[2019.08]
idx: 7, color: 25276781, day: 4
idx: 14, color: 25276781, day: 11
idx: 18, color:

[2016.08]
idx: 7, color: 25276781, day: 7
idx: 14, color: 25276781, day: 14
idx: 15, color: 25276781, day: 15
idx: 21, color: 25276781, day: 21
idx: 28, color: 25276781, day: 28
[2016.07]
idx: 7, color: 25276781, day: 3
idx: 14, color: 25276781, day: 10
idx: 21, color: 25276781, day: 17
idx: 28, color: 25276781, day: 24
idx: 35, color: 25276781, day: 31
[2016.06]
idx: 7, color: 25276781, day: 5
idx: 8, color: 25276781, day: 6
idx: 14, color: 25276781, day: 12
idx: 21, color: 25276781, day: 19
idx: 28, color: 25276781, day: 26
[2016.05]
idx: 0, color: 25276781, day: 1
idx: 4, color: 25276781, day: 5
idx: 5, color: 25276781, day: 6
idx: 7, color: 25276781, day: 8
idx: 13, color: 25276781, day: 14
idx: 14, color: 25276781, day: 15
idx: 21, color: 25276781, day: 22
idx: 28, color: 25276781, day: 29
[2016.04]
idx: 7, color: 25276781, day: 3
idx: 14, color: 25276781, day: 10
idx: 17, color: 25276781, day: 13
idx: 21, color: 25276781, day: 17
idx: 28, color: 25276781, day: 24
[2016.03]
idx: 2

idx: 35, color: 25276781, day: 31
[2013.02]
idx: 7, color: 25276781, day: 3
idx: 13, color: 25276781, day: 9
idx: 14, color: 25276781, day: 10
idx: 15, color: 25276781, day: 11
idx: 21, color: 25276781, day: 17
idx: 28, color: 25276781, day: 24
[2013.01]
idx: 2, color: 25276781, day: 1
idx: 7, color: 25276781, day: 6
idx: 14, color: 25276781, day: 13
idx: 21, color: 25276781, day: 20
idx: 28, color: 25276781, day: 27
[2012.12]
idx: 7, color: 25276781, day: 2
idx: 14, color: 25276781, day: 9
idx: 21, color: 25276781, day: 16
idx: 24, color: 25276781, day: 19
idx: 28, color: 25276781, day: 23
idx: 30, color: 25276781, day: 25
idx: 35, color: 25276781, day: 30
[2012.11]
idx: 7, color: 25276781, day: 4
idx: 14, color: 25276781, day: 11
idx: 21, color: 25276781, day: 18
idx: 28, color: 25276781, day: 25
[2012.10]
idx: 1, color: 25276781, day: 1
idx: 3, color: 25276781, day: 3
idx: 7, color: 25276781, day: 7
idx: 14, color: 25276781, day: 14
idx: 21, color: 25276781, day: 21
idx: 28, color: 

In [6]:
holiday_list[:5]

[datetime.datetime(2023, 6, 4, 0, 0),
 datetime.datetime(2023, 6, 6, 0, 0),
 datetime.datetime(2023, 6, 11, 0, 0),
 datetime.datetime(2023, 6, 18, 0, 0),
 datetime.datetime(2023, 6, 25, 0, 0)]

---
# 2. Saturday
- 네이버달력에서 토요일은 검색으로 나타나므로 별도로 추가

In [7]:
date_list = pd.date_range(start=start_date, end=end_date, freq='D')
date_list[:5]

DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03', '2012-01-04',
               '2012-01-05'],
              dtype='datetime64[ns]', freq='D')

In [8]:
weekend_list = [i for i in date_list if i.weekday() > 4] # 5: 토요일, 6: 일요일
weekend_list[:5]

[Timestamp('2012-01-01 00:00:00', freq='D'),
 Timestamp('2012-01-07 00:00:00', freq='D'),
 Timestamp('2012-01-08 00:00:00', freq='D'),
 Timestamp('2012-01-14 00:00:00', freq='D'),
 Timestamp('2012-01-15 00:00:00', freq='D')]

---
# 3. 병합

## 3-1. preprocess
- datetime으로 병합해주기

In [37]:
def preprocess(list):
    return [datetime.datetime.strptime(i.strftime("%y-%m-%d"), '%y-%m-%d') for i in list] 
#     return [i.strf0time("%y-%m-%d") for i in list] 

# strftime: datetime, timestamp -> str, strptime: str -> datetime

In [38]:
holiday_list_prep = preprocess(holiday_list)
weekend_list_prep = preprocess(weekend_list)

In [39]:
len(holiday_list_prep)

774

In [40]:
len(weekend_list_prep)

1199

In [41]:
len(holiday_list_prep + weekend_list_prep)

1973

In [42]:
len(set(holiday_list_prep + weekend_list_prep))

1361

In [43]:
final_list = sorted(set(holiday_list_prep + weekend_list_prep), reverse=True)
final_list

[datetime.datetime(2023, 6, 25, 0, 0),
 datetime.datetime(2023, 6, 24, 0, 0),
 datetime.datetime(2023, 6, 18, 0, 0),
 datetime.datetime(2023, 6, 17, 0, 0),
 datetime.datetime(2023, 6, 11, 0, 0),
 datetime.datetime(2023, 6, 10, 0, 0),
 datetime.datetime(2023, 6, 6, 0, 0),
 datetime.datetime(2023, 6, 4, 0, 0),
 datetime.datetime(2023, 6, 3, 0, 0),
 datetime.datetime(2023, 5, 29, 0, 0),
 datetime.datetime(2023, 5, 28, 0, 0),
 datetime.datetime(2023, 5, 27, 0, 0),
 datetime.datetime(2023, 5, 21, 0, 0),
 datetime.datetime(2023, 5, 20, 0, 0),
 datetime.datetime(2023, 5, 14, 0, 0),
 datetime.datetime(2023, 5, 13, 0, 0),
 datetime.datetime(2023, 5, 7, 0, 0),
 datetime.datetime(2023, 5, 6, 0, 0),
 datetime.datetime(2023, 5, 5, 0, 0),
 datetime.datetime(2023, 4, 30, 0, 0),
 datetime.datetime(2023, 4, 29, 0, 0),
 datetime.datetime(2023, 4, 23, 0, 0),
 datetime.datetime(2023, 4, 22, 0, 0),
 datetime.datetime(2023, 4, 16, 0, 0),
 datetime.datetime(2023, 4, 15, 0, 0),
 datetime.datetime(2023, 4, 9, 

# 4.  BussniessDay Classifier 함수화

In [44]:
# 같은 기간동안의 연속적인 날짜 적용
naive_sample = sorted(pd.date_range(start=start_date, end=end_date, freq='D'), reverse=True)
naive_sample_df = pd.DataFrame(naive_sample, columns=['Date'])
naive_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4199 entries, 0 to 4198
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    4199 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 32.9 KB


In [65]:
# naive_sample_df.iloc[:,:1]

,Date
0,2023-06-30
1,2023-06-29
2,2023-06-28
3,2023-06-27
4,2023-06-26
...,...
4193,2012-01-06
4194,2012-01-05
4195,2012-01-04
4196,2012-01-03


In [45]:
naive_sample_df.head()

,Date
0,2023-06-30
1,2023-06-29
2,2023-06-28
3,2023-06-27
4,2023-06-26


In [99]:
def BussniessDayClassifier(df, list):
    for idx, v in enumerate(df['Date']): # 가장맨 앞에 Date
        if v in list:
            df.drop(idx, inplace=True)
            
    return df

In [102]:
BussniessDayClassifier(naive_sample_df, final_list)

,Date
0,2023-06-30
1,2023-06-29
2,2023-06-28
3,2023-06-27
4,2023-06-26
...,...
4193,2012-01-06
4194,2012-01-05
4195,2012-01-04
4196,2012-01-03


In [101]:
for i in naive_sample_df['Date'][:50]:
    print(i)

2023-06-30 00:00:00
2023-06-29 00:00:00
2023-06-28 00:00:00
2023-06-27 00:00:00
2023-06-26 00:00:00
2023-06-23 00:00:00
2023-06-22 00:00:00
2023-06-21 00:00:00
2023-06-20 00:00:00
2023-06-19 00:00:00
2023-06-16 00:00:00
2023-06-15 00:00:00
2023-06-14 00:00:00
2023-06-13 00:00:00
2023-06-12 00:00:00
2023-06-09 00:00:00
2023-06-08 00:00:00
2023-06-07 00:00:00
2023-06-05 00:00:00
2023-06-02 00:00:00
2023-06-01 00:00:00
2023-05-31 00:00:00
2023-05-30 00:00:00
2023-05-26 00:00:00
2023-05-25 00:00:00
2023-05-24 00:00:00
2023-05-23 00:00:00
2023-05-22 00:00:00
2023-05-19 00:00:00
2023-05-18 00:00:00
2023-05-17 00:00:00
2023-05-16 00:00:00
2023-05-15 00:00:00
2023-05-12 00:00:00
2023-05-11 00:00:00
2023-05-10 00:00:00
2023-05-09 00:00:00
2023-05-08 00:00:00
2023-05-04 00:00:00
2023-05-03 00:00:00
2023-05-02 00:00:00
2023-05-01 00:00:00
2023-04-28 00:00:00
2023-04-27 00:00:00
2023-04-26 00:00:00
2023-04-25 00:00:00
2023-04-24 00:00:00
2023-04-21 00:00:00
2023-04-20 00:00:00
2023-04-19 00:00:00


# 5. 실제데이터 응용

In [114]:
raw_data = pd.read_excel('./Fra krw_bloomberg.xlsx')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        3488 non-null   datetime64[ns]
 1   1x4         3488 non-null   float64       
 2   Unnamed: 2  0 non-null      float64       
 3   Date.1      3960 non-null   datetime64[ns]
 4   3x6         3959 non-null   float64       
 5   Unnamed: 5  0 non-null      float64       
 6   Date.2      3721 non-null   datetime64[ns]
 7   6x9         3720 non-null   float64       
 8   Unnamed: 8  0 non-null      float64       
 9   Date.3      3814 non-null   datetime64[ns]
 10  9x12        3813 non-null   float64       
dtypes: datetime64[ns](4), float64(7)
memory usage: 340.4 KB


위 예제의 end_date(2020년)기준 맞춰주기

In [115]:
# raw_data[:852]
# raw_data = raw_data[:852]

In [116]:
raw_data14 = raw_data[['Date', '1x4']]
raw_data36 = raw_data[['Date.1', '3x6']]
raw_data69 = raw_data[['Date.2', '6x9']]
raw_data912 = raw_data[['Date.3', '9x12']]

In [117]:
columns = ['Date', 'value']

raw_data14.columns = columns
raw_data36.columns = columns
raw_data69.columns = columns
raw_data912.columns = columns

In [118]:
print(f'''raw_data14: {len(raw_data14)}
raw_data36: {len(raw_data36)}
raw_data69: {len(raw_data69)}
raw_data912: {len(raw_data912)}''')

raw_data14: 3960
raw_data36: 3960
raw_data69: 3960
raw_data912: 3960


In [112]:
BussniessDayClassifier(raw_data14, final_list)
BussniessDayClassifier(raw_data36, final_list)
BussniessDayClassifier(raw_data69, final_list)
BussniessDayClassifier(raw_data912, final_list)

C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppD

C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppD

C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppD

C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppD

C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppD

C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppD

C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppD

C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppD

C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppData\Local\Temp\ipykernel_40052\1411572517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(idx, inplace=True)
C:\Users\jjgus\AppD

,Date,value
0,2023-06-12,NaN
1,2023-06-09,3.60
2,2023-06-08,3.68
3,2023-06-07,3.58
4,2023-06-05,3.60
...,...,...
3955,NaT,NaN
3956,NaT,NaN
3957,NaT,NaN
3958,NaT,NaN


In [113]:
print(f'''raw_data14: {len(raw_data14)}
raw_data36: {len(raw_data36)}
raw_data69: {len(raw_data69)}
raw_data912: {len(raw_data912)}''')

raw_data14: 3942
raw_data36: 3849
raw_data69: 3849
raw_data912: 3942
